# Network diagrams for NL/GQL

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

import networkx as nwkx
import collections

In [ ]:
# get all modes in an nx-ny grid
def modes(nx,ny):
        
    i = 0
    x = []
    for p in np.arange(-nx,nx+1):
        for q in np.arange(-ny,ny+1):

            x.append((p,q))
            i += 1
    
    return x

# get all triads in an nx-ny grid
def triads(nx,ny):
        
    x = []
    for p in modes(nx,ny):
        for q in modes(nx,ny):
                        
            k = (p[0]+q[0],p[1]+q[1])
            
            if not(k[0]==0 and k[1]==0) and not(p[0]==0 and p[1]==0) and not(q[0]==0 and q[1]==0): # non-zeros
          
                if np.abs(k[0])<=nx and np.abs(k[1])<=ny: # remove out of bounds
                   
                    if p != q and q != k: # no self-interactions
                    
                        x.append((p,q,k)) # pxq -> k
    
    return x

# list edges between modes and triads
def edges(nx,ny,A):
        
    m = modes(nx,ny)
    
    e = []
    for t in triads(nx,ny):
        
        i = m.index(t[0])
        j = m.index(t[1])
        
        if(A[i,j] > 0.0):
            e.append((t[0],t))
            e.append((t[1],t))
            e.append((t[2],t))
    
    
    return e

# network graph using modes,edges,triads
def graph(nx,ny,A):

    m = modes(nx,ny)
    m.remove((0,0))

    t = triads(nx,ny)
    e = edges(nx,ny,A)

    bpg=nwkx.Graph()

    bpg.add_nodes_from(m,bipartite=0)
    bpg.add_nodes_from(t,bipartite=1)
    bpg.add_edges_from(e)

    dl = list(nwkx.connected_components(bpg))[1:]
    for d in dl:
        for n in d:
            bpg.remove_node(n)

    return bpg

# degree distribution on mode set f of graph G
def degreedist(f,G):
    
    d = G.degree(f) 
    v = sorted([d[1] for d in d])
    h = [v.count(x) for x in v]

    return v,h

# Diagnostics

# print(modes(1,1))
# print(len(triads(1,1)))
# print(triads(1,1))
# ((0,1),(0,-1),(0,0)) in triads(1,1)

# m = modes(1,1)[3]

# print("Looking for mode: ", m)

# for t in triads(1,1):
#     if(m in t):
#         print("Present in triad:", t)

# A = np.zeros((9,9))
# A[2,3] = 1.0
# e = edges(1,1,A)

# print(e)

In [ ]:
# set positions of modes in network
def nodepos(G):
    
    l, r = nwkx.bipartite.sets(G)
    pos = {}

    pos.update((node, (0, index)) for index, node in enumerate(l))
    pos.update((node, (1, index)) for index, node in enumerate(r))

    return pos

# draw adjacencies with ticks
def ax_adj(ax,nx,ny):
    
    l = []
    for m in modes(nx,ny):
        l.append(str(m))
    
    ax.set_xticks(np.arange(0,(2*nx+1)*(2*ny+1),1))
    ax.set_yticks(np.arange(0,(2*nx+1)*(2*ny+1),1))
    ax.set_xticklabels(l,fontsize=14)
    ax.set_yticklabels(l,fontsize=14)
    
# Graph diagnostics

# A = np.ones((9,9))
# G = graph(1,1,A)
# f,g = nwkx.bipartite.sets(G)

# pos = nodepos(G)

# fig,ax = plt.subplots(figsize=(5,5))
# nwkx.draw(G,pos=pos,node_size=25,with_labels=True,width=0.75)
# plt.show()

# fig,ax = plt.subplots(1,2,figsize=(9,4))

# mdegs = G.degree(f) # dictionary node:degree 
# mvals = sorted([d[1] for d in mdegs])
# mhist = [mvals.count(x) for x in mvals]
# ax[0].plot(mvals,mhist,'o-')
# ax[0].set_xlabel('Degree')
# ax[0].set_ylabel('Number of modes')

# mdegs = G.degree(g) # dictionary node:degree 
# mvals = sorted([d[1] for d in mdegs])
# mhist = [mvals.count(x) for x in mvals]
# ax[1].plot(mvals,mhist,'o-')
# ax[1].set_xlabel('Degree')
# ax[1].set_ylabel('Number of triads')

# plt.show()

In [ ]:
dn = "data/"

## 2x2 Grids

In [ ]:
nx,ny = 2,2

# NL
fn = dn+"adjacency_nl_2x2.npz"
nl = np.load(fn,allow_pickle=True) 
# QL
fn = dn+"adjacency_ql_2x2.npz"
ql = np.load(fn,allow_pickle=True)

### 2x2 NL

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,4))

im = ax[0].imshow(np.abs(nl['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$|B_{mn}|$',fontsize=15)
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.angle(nl['B']),cmap="RdBu_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'$\angle B_{mn}$',fontsize=15)
ax_adj(ax[1],nx-1,ny-1)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,4))

im = ax[0].imshow(np.abs(nl['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$|B_{mn}|$',fontsize=15)
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.angle(nl['B']),cmap="RdBu_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'$\angle B_{mn}$',fontsize=15)
ax_adj(ax[1],nx-1,ny-1)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,4))

im = ax[0].imshow(np.abs(nl['C']),cmap="Reds",origin="lower",interpolation="none",norm=mpl.colors.LogNorm())
ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.angle(nl['C']),cmap="RdBu_r",origin="lower",interpolation="none")
ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])
ax_adj(ax[1],nx-1,ny-1)

plt.show()

In [ ]:
# draw graph
G = graph(nx-1,ny-1,nl['C'])

fig,ax = plt.subplots(figsize=(6,6))
nwkx.draw(G,pos=nodepos(G),node_size=25,with_labels=True,width=0.75)
plt.show()

### 2x2 QL

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,4))

im = ax[0].imshow(np.abs(ql['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$|B_{mn}|$',fontsize=15)
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.angle(ql['B']),cmap="RdBu_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'$\angle B_{mn}$',fontsize=15)
ax_adj(ax[1],nx-1,ny-1)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,4))

im = ax[0].imshow(np.abs(ql['C']),cmap="Reds",origin="lower",interpolation="none",norm=mpl.colors.LogNorm())
ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.angle(ql['C']),cmap="RdBu_r",origin="lower",interpolation="none")
ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])
ax_adj(ax[1],nx-1,ny-1)

plt.show()

In [ ]:
# draw graph
G = graph(nx-1,ny-1,ql['C'])

fig,ax = plt.subplots(figsize=(6,6))
nwkx.draw(G,pos=nodepos(G),node_size=25,with_labels=True,width=0.75)
plt.show()

> I think QL and NL must match in this case.

## 3x3 Grid Tests

In [ ]:
nx,ny = 3,3

# NL
fn = dn+"adjacency_nl_3x3.npz"
nl = np.load(fn,allow_pickle=True) 

fn = dn+"adjacency_ql_3x3.npz"
ql = np.load(fn,allow_pickle=True) 

fn = dn+"adjacency_gql1_3x3.npz"
gql = np.load(fn,allow_pickle=True) 

### Linear Terms

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |B_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| B_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| B_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

### Nonlinear terms

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['C']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |C_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['C']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| C_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['C']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| C_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

> here GQL(1) looks identical to QL for some reason. This can happen sometmes.

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.angle(nl['C']),cmap="RdBu_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: \angle C_{mn}$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.angle(ql['C']),cmap="RdBu_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $\angle C_{mn} $')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.angle(gql['C']),cmap="RdBu_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $\angle C_{mn} $')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

### Networks

In [ ]:
# draw graph
G1 = graph(nx-1,ny-1,nl['C'])
G2 = graph(nx-1,ny-1,ql['C'])
G3 = graph(nx-1,ny-1,gql['C'])

f1,g1 = nwkx.bipartite.sets(G1)
f2,g2 = nwkx.bipartite.sets(G2)
f3,g3 = nwkx.bipartite.sets(G3)

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,4))

ax[0].set_title('NL')
nwkx.draw(G1,pos=nodepos(G1),node_size=25,with_labels=False,width=0.75,ax=ax[0])

ax[1].set_title('QL')
nwkx.draw(G2,pos=nodepos(G2),node_size=25,with_labels=False,width=0.75,ax=ax[1])

ax[2].set_title('GQL(1)')
nwkx.draw(G3,pos=nodepos(G3),node_size=25,with_labels=False,width=0.75,ax=ax[2])

plt.show()

> Double check why QL and GQL(1) are identical 

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(10,3))

vf,hf = degreedist(f1,G1)
ax[0].plot(vf,hf,'o-')
ax[0].set_title('NL')

vf,hf = degreedist(f2,G2)
ax[1].plot(vf,hf,'o-')
ax[1].set_title('QL')

vf,hf = degreedist(f3,G3)
ax[2].plot(vf,hf,'o-')
ax[2].set_title('GQL(1)')

ax[0].set_xlabel('Degree')
ax[0].set_ylabel('Number of modes')

plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(10,3))

vf,hf = degreedist(g1,G1)
ax[0].plot(vf,hf,'o-')
ax[0].set_title('NL')

vf,hf = degreedist(g2,G2)
ax[1].plot(vf,hf,'o-')
ax[1].set_title('QL')

vf,hf = degreedist(g3,G3)
ax[2].plot(vf,hf,'o-')
ax[2].set_title('GQL(1)')

ax[0].set_xlabel('Degree')
ax[0].set_ylabel('Number of triads')

plt.show()

## 6x6 grid tests

In [ ]:
nx,ny = 6,6

fn = dn+"adjacency_nl_6x6.npz"
nl = np.load(fn,allow_pickle=True) 

fn = dn+"adjacency_ql_6x6.npz"
ql = np.load(fn,allow_pickle=True) 

fn = dn+"adjacency_gql1_6x6.npz"
gql = np.load(fn,allow_pickle=True) 

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |B_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| B_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| B_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['C']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |C_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['C']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| C_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['C']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| C_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['C']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |C_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['C']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| C_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['C']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| C_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

> GQL(1) clearly is richer than QL 

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.angle(nl['C']),cmap="RdBu_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: \angle C_{mn}$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.angle(ql['C']),cmap="RdBu_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $\angle C_{mn} $')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.angle(gql['C']),cmap="RdBu_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $\angle C_{mn} $')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

In [ ]:
# draw graph
G1 = graph(nx-1,ny-1,nl['C'])
G2 = graph(nx-1,ny-1,ql['C'])
G3 = graph(nx-1,ny-1,gql['C'])

f1,g1 = nwkx.bipartite.sets(G1)
f2,g2 = nwkx.bipartite.sets(G2)
f3,g3 = nwkx.bipartite.sets(G3)

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,4))

ax[0].set_title('NL')
nwkx.draw(G1,pos=nodepos(G1),node_size=25,with_labels=False,width=0.75,ax=ax[0])

ax[1].set_title('QL')
nwkx.draw(G2,pos=nodepos(G2),node_size=25,with_labels=False,width=0.75,ax=ax[1])

ax[2].set_title('GQL(1)')
nwkx.draw(G3,pos=nodepos(G3),node_size=25,with_labels=False,width=0.75,ax=ax[2])

plt.show()

> Hard to see!

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(10,3))

vf,hf = degreedist(f1,G1)
ax[0].plot(vf,hf,'o-')
ax[0].set_title('NL')

vf,hf = degreedist(f2,G2)
ax[1].plot(vf,hf,'o-')
ax[1].set_title('QL')

vf,hf = degreedist(f3,G3)
ax[2].plot(vf,hf,'o-')
ax[2].set_title('GQL(1)')

ax[0].set_xlabel('Degree')
ax[0].set_ylabel('Number of modes')

plt.show()

> interesting changes visible through degree distributions

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(10,3))

vf,hf = degreedist(g1,G1)
ax[0].plot(vf,hf,'o-')
ax[0].set_title('NL')

vf,hf = degreedist(g2,G2)
ax[1].plot(vf,hf,'o-')
ax[1].set_title('QL')

vf,hf = degreedist(g3,G3)
ax[2].plot(vf,hf,'o-')
ax[2].set_title('GQL(1)')

ax[0].set_xlabel('Degree')
ax[0].set_ylabel('Number of triads')

plt.show()

## 8x8 grids

In [ ]:
nx,ny = 8,8

fn = dn+"adjacency_nl_8x8.npz"
nl = np.load(fn,allow_pickle=True) 

fn = dn+"adjacency_ql_8x8.npz"
ql = np.load(fn,allow_pickle=True) 

fn = dn+"adjacency_gql1_8x8.npz"
gql = np.load(fn,allow_pickle=True) 

### Forcing terms

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['A']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |A_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['A']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| A_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['A']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| A_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

### Linear terms

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |B_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| B_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| B_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

### Nonlinear terms

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['C']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |C_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['C']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| C_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['C']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| C_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

## 8x8 Grids (solution-based)

#### Single mode forced

In [ ]:
nl_sol = np.load(dn+"nl_sol_m53_e001_8x8.npz",allow_pickle=True) 
nl = np.load(dn+"nl_adj_m53_e001_8x8.npz",allow_pickle=True) 

ql_sol = np.load(dn+"ql_sol_m53_e001_8x8.npz",allow_pickle=True) 
ql = np.load(dn+"ql_adj_m53_e001_8x8.npz",allow_pickle=True) 

gql_sol = np.load(dn+"gql1_sol_m53_e001_8x8.npz",allow_pickle=True) 
gql = np.load(dn+"gql1_adj_m53_e001_8x8.npz",allow_pickle=True) 

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(13,4))

ax[0].set_title('NL')
ax[0].plot(nl_sol['t'],nl_sol['Emt'][0],'k',label=0)
for i,x in enumerate(nl_sol['Emt'][1:]):
    ax[0].plot(nl_sol['t'],x,label=i+1)

ax[1].set_title('QL')
ax[1].plot(ql_sol['t'],ql_sol['Emt'][0],'k',label=0)
for i,x in enumerate(ql_sol['Emt'][1:]):
    ax[1].plot(ql_sol['t'],x,label=i+1)

ax[2].set_title('GQL(1)')
ax[2].plot(gql_sol['t'],gql_sol['Emt'][0],'k',label=0)
for i,x in enumerate(gql_sol['Emt'][1:]):
    ax[2].plot(gql_sol['t'],x,label=i+1)

for a in ax:
    a.set_yscale('log')
    a.set_ylim(1e-12,1e0)
    
ax[0].set_ylabel(r'$E_m$')
ax[1].legend()
plt.show()

> shows energy in zonal wavenumbers

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(13,4))

ax[0].set_title('NL')
ax[0].imshow(nl_sol['F'])

ax[1].set_title('QL')
ax[1].imshow(ql_sol['F'])

ax[2].set_title('GQL(1)')
ax[2].imshow(gql_sol['F'])

plt.show()

> forced modes

### Forcing terms

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['A']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |A_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['A']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| A_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['A']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| A_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

> literally one triad

### Linear terms

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |B_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| B_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| B_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

> here we can already see the localisation in QL, and non-locality in GQL

### Nonlinear terms

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |C_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| C_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| C_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

> GQL supports a whole band, rather than a localised region of triads as in QL

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['Cl']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |C_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['Cl']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| C_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['Cl']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| C_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

> interactions resulting in low modes. Makes a lot of sense - in NL, all interactions lead to low modes as all modes are low modes. In QL only HH interactions lead to low modes (the mean), and in GQL there is a band diagonal of triads where low modes are produced.

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['Ch']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |C_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['Ch']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| C_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['Ch']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| C_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

> interactions leading to high modes. Again makes sense because NL has no such interactions. QL involves interactions with the mean (hence along the horizontal). And GQL involves high modes interacting with mean and zonal wavenumber unity.

#### narrow-band forcing

In [ ]:
nl_sol = np.load(dn+"nl_sol_e001_8x8.npz",allow_pickle=True) 
nl = np.load(dn+"nl_adj_e001_8x8.npz",allow_pickle=True) 

ql_sol = np.load(dn+"ql_sol_e001_8x8.npz",allow_pickle=True) 
ql = np.load(dn+"ql_adj_e001_8x8.npz",allow_pickle=True) 

gql_sol = np.load(dn+"gql1_sol_e001_8x8.npz",allow_pickle=True) 
gql = np.load(dn+"gql1_adj_e001_8x8.npz",allow_pickle=True) 

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(13,4))

ax[0].set_title('NL')
ax[0].plot(nl_sol['t'],nl_sol['Emt'][0],'k',label=0)
for i,x in enumerate(nl_sol['Emt'][1:]):
    ax[0].plot(nl_sol['t'],x,label=i+1)

ax[1].set_title('QL')
ax[1].plot(ql_sol['t'],ql_sol['Emt'][0],'k',label=0)
for i,x in enumerate(ql_sol['Emt'][1:]):
    ax[1].plot(ql_sol['t'],x,label=i+1)

ax[2].set_title('GQL(1)')
ax[2].plot(gql_sol['t'],gql_sol['Emt'][0],'k',label=0)
for i,x in enumerate(gql_sol['Emt'][1:]):
    ax[2].plot(gql_sol['t'],x,label=i+1)

for a in ax:
    a.set_yscale('log')
    a.set_ylim(1e-12,1e0)
    
ax[0].set_ylabel(r'$E_m$')
ax[1].legend()
plt.show()

> shows energy in zonal wavenumbers

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(13,4))

ax[0].set_title('NL')
ax[0].imshow(nl_sol['F'])

ax[1].set_title('QL')
ax[1].imshow(ql_sol['F'])

ax[2].set_title('GQL(1)')
ax[2].imshow(gql_sol['F'])

plt.show()

> forced modes

### Forcing terms

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['A']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |A_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['A']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| A_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['A']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| A_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

### Linear terms

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |B_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| B_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['B']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| B_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

### Nonlinear terms

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |C_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| C_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| C_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

> this is a band type forcing - forcing all modes wi"thin a zonal wavenumber

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['Cl']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |C_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['Cl']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| C_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['Cl']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| C_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,3))

im = ax[0].imshow(np.abs(nl['Ch']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$NL: |C_{mn}|$')
ax_adj(ax[0],nx-1,ny-1)

im = ax[1].imshow(np.abs(ql['Ch']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'QL: $| C_{mn} |$')
ax_adj(ax[1],nx-1,ny-1)

im = ax[2].imshow(np.abs(gql['Ch']),cmap="nipy_spectral_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[2])
ax[2].set_title(r'GQL(1): $| C_{mn} |$')
ax_adj(ax[2],nx-1,ny-1)

plt.show()